In [2]:
import pandas as pd
import duckdb

In [3]:
CPI25 = pd.read_csv('PCPI25M2.csv')
CPI25

,DATE,CPI
0,1947:01,21.5
1,1947:02,21.6
2,1947:03,22.0
3,1947:04,22.0
4,1947:05,22.0
...,...,...
932,2024:09,314.9
933,2024:10,315.6
934,2024:11,316.4
935,2024:12,317.6


In [4]:
file = 'Fed_CPI2.db'

In [5]:
#trunc method
def trunc_and_load(con,data):
    # we simply truncate the table and load the new data
    con.sql(("CREATE OR REPLACE TABLE cpi_trunc AS SELECT * FROM data"))
with duckdb.connect(file) as con:
    trunc_and_load(con, CPI25)
    print(con.sql('SELECT * FROM cpi_trunc').fetchdf()) 



        DATE    CPI
0    1947:01   21.5
1    1947:02   21.6
2    1947:03   22.0
3    1947:04   22.0
4    1947:05   22.0
..       ...    ...
932  2024:09  314.9
933  2024:10  315.6
934  2024:11  316.4
935  2024:12  317.6
936  2025:01  319.1

[937 rows x 2 columns]


In [ ]:
#drop mistaken addition to db
with duckdb.connect(file) as con:
    con.sql('DROP TABLE IF EXISTS data_trunc')

In [ ]:
#append method

In [9]:
#inc method--inspired by Chat GPT
def incremental_load(con, new_data):
    # Load existing CPI data from table into a df
    existing_cpi = con.sql("SELECT * FROM cpi_inc").fetchdf()
    # Merge existing and new data to compare
    merged_cpi = existing_cpi.merge(new_data, on="DATE", how="outer", suffixes=("_old", "_new"))
    # Identify records to update (existing dates with revised values)
    updates = merged_cpi.dropna(subset=["CPI_old", "CPI_new"])
    updates = updates[updates["CPI_old"] != updates["CPI_new"]][["DATE", "CPI_new"]]
    # Identify new records to insert (dates not in the existing table)
    inserts = merged_cpi[merged_cpi["CPI_old"].isna()][["DATE", "CPI_new"]]
    # Update existing records
    for _, row in updates.iterrows():
        con.sql(f"UPDATE cpi_inc SET CPI = {row['CPI_new']} WHERE DATE = '{row['DATE']}'")
    # Insert new records
    if not inserts.empty:
        con.sql("INSERT INTO cpi_inc SELECT * FROM inserts")

with duckdb.connect(file) as con:
    con.sql('BEGIN TRANSACTION') # starting a transaction -- changes are synced once = improves performance
    incremental_load(con, CPI25)
    con.sql('COMMIT') # committing the transaction 
    print(con.sql('SELECT * FROM cpi_inc').fetchdf())

        DATE    CPI
0    1947:01   21.5
1    1947:02   21.6
2    1947:03   22.0
3    1947:04   22.0
4    1947:05   22.0
..       ...    ...
932  2024:09  314.9
933  2024:10  315.6
934  2024:11  316.4
935  2024:12  317.6
936  2025:01  319.1

[937 rows x 2 columns]
